## **Instalação do pacote deap**

In [78]:
! pip install deap

## **Importação do framework e bibliotecas**

In [79]:
import random

from deap import base
from deap import creator
from deap import tools
from deap import gp
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [80]:
import numpy as np
import operator

In [81]:
import matplotlib.pyplot as plt   # plotting

Carregando as imagens

In [82]:
# Carregar o conjunto de dados MNIST reduzido
digits = load_digits()
X, y = digits.images, digits.target

n_samples = len(X)
X = X.reshape((n_samples, -1))  # Achatar as imagens em vetores

# Normalizar os dados para melhorar o desempenho
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Criar rótulos binários (pares = 0, ímpares = 1)
labels = np.where(y % 2 == 0, 0, 1)

In [83]:
features = np.array([
   [np.sum(img),                 # Soma dos pixels
    np.count_nonzero(img),       # Número de pixels não nulos
    np.mean(img),                # Média da intensidade
    np.std(img),                 # Desvio padrão da intensidade
    np.count_nonzero(img) / 64   # Proporção de pixels ativos
   ]
   for img in X
])

Geração dos Dados para o exemplo

Criação do Fitness e Individuo

In [84]:
# Criar o problema de maximização (quanto mais acertos, melhor)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximizar acurácia
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Criação da função que cria o indivíduo com sigma embutido no cromossomo

In [85]:
# Definir os operadores e terminais
pset = gp.PrimitiveSet("MAIN", 5)  # Duas variáveis de entrada (x e y)
# pset.renameArguments(ARG0='x')
# pset.renameArguments(ARG1='y')
pset.renameArguments(ARG0='sum_pixels', ARG1='nonzero_pixels', ARG2='mean_pixels', ARG3='std_pixels', ARG4='active_ratio')

# Operações matemáticas
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)

# Funções lógicas
pset.addPrimitive(operator.lt, 2)  # Menor que
pset.addPrimitive(operator.gt, 2)  # Maior que

# Constantes
pset.addTerminal(1)
pset.addTerminal(2)

# pset.addPrimitive(operator.truediv, 2)
# pset.addPrimitive(if_then_else, 3)
# pset.addEphemeralConstant("rand101", lambda: random.uniform(-1, 1))



# Divisão protegida para evitar divisão por zero
def protectedDiv(left, right):
    # Evita divisão por valores muito pequenos
    if abs(right) < 1e-6:
        return 1  # Retorna 1 se o denominador for zero ou muito pequeno
    return left / right

pset.addPrimitive(protectedDiv, 2)

# Configuração do toolbox
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Compilar as árvores para transformá-las em funções executáveis
toolbox.register("compile", gp.compile, pset=pset)

Função de avaliação

In [86]:
def evalDigits(individual):
   func = toolbox.compile(expr=individual)
   correct = 0
   for (sum_pixels, nonzero_pixels, mean_pixels, std_pixels,
active_ratio), label in zip(features, labels):
       try:
           prediction = func(sum_pixels, nonzero_pixels, mean_pixels,
std_pixels, active_ratio)
           predicted_label = 1 if prediction > 0 else 0
       except:
           predicted_label = 0  # Penaliza funções que dão erro

       if predicted_label == label:
           correct += 1
   return correct / len(features),  # Retorna a acurácia

Configurar operadores

In [87]:
toolbox.register("evaluate", evalDigits)
toolbox.register("select", tools.selTournament, tournsize=5)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)

# Limitar o tamanho das árvores para evitar crescimento descontrolado (bloat)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

Algoritmo Evolutivo

In [88]:
from deap import algorithms

In [90]:
# Inicializar a população
population = toolbox.population(n=500)
hof = tools.HallOfFame(1)  # Guardar o melhor indivíduo

# Estatísticas
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)

# Rodar o GP por 40 gerações
population, log = algorithms.eaSimple(population, toolbox,
                                      cxpb=0.5, mutpb=0.3,
                                      ngen=40, stats=stats,
                                      halloffame=hof, verbose=True)


gen	nevals	avg     	max     
0  	500   	0.496055	0.538676
1  	325   	0.508997	0.538676
2  	330   	0.519876	0.542014
3  	317   	0.527073	0.542014
4  	343   	0.528003	0.543127
5  	322   	0.529357	0.543127
6  	327   	0.527763	0.543127
7  	304   	0.528093	0.548136
8  	332   	0.528185	0.548136
9  	321   	0.531495	0.548136
10 	323   	0.532835	0.548136
11 	344   	0.532189	0.549805
12 	319   	0.532595	0.549805
13 	342   	0.527316	0.549805
14 	334   	0.530727	0.549805
15 	314   	0.530503	0.549805
16 	337   	0.528668	0.549805
17 	338   	0.526934	0.549805
18 	323   	0.527469	0.549805
19 	321   	0.528272	0.549805
20 	306   	0.530615	0.549805
21 	300   	0.531151	0.549805
22 	333   	0.528824	0.549805
23 	308   	0.528299	0.549805
24 	328   	0.529665	0.549805
25 	328   	0.529628	0.549805
26 	339   	0.528154	0.549805
27 	323   	0.530359	0.549805
28 	307   	0.531383	0.549805
29 	338   	0.530028	0.549805
30 	307   	0.53307 	0.549805
31 	325   	0.530659	0.549805
32 	320   	0.53053 	0.549805
33 	329   	0.5